In [1]:
from argparse import ArgumentParser
import psutil as ps
from time import sleep, time
import uuid
import paho.mqtt.client as mqtt
import json
import redis

In [2]:
#Create redis client
REDIS_HOST='redis-19577.c81.us-east-1-2.ec2.cloud.redislabs.com'
REDIS_PORT = 19577
REDIS_USER = 'default'
REDIS_PASSWORD = 'so7BaeUMztM1pEpKusnIWrk9PN0nSVdm'

redis_client = redis.Redis(
    host = REDIS_HOST,
    port = REDIS_PORT,
    username = REDIS_USER,
    password = REDIS_PASSWORD
    )

#Check if it is connected
is_connected = redis_client.ping()
print(is_connected)



True


In [3]:
#Create the mqtt client
client =  mqtt.Client()

In [4]:
topic = 's303873'

#Callbacks
def on_connect(client, userdata, flags, rc): #triggered by connection
    #subscribe for a topic
    client.subscribe(topic)

def on_message(client, userdata, msg):       #triggered by received message
    #Take the message payload
    # decode it since it is binary
    message = msg.payload.decode()
    #Get the message topic
    topic = msg.topic
    print(f'Message received :\n {message}\nFrom: {topic}')
    #Convert the msg payload into a dictionary
    message = json.loads(message)
    mac_address = message['mac_address']
    timestamp_in_ms = message['timestamp']
    battery_level = message['battery_level']
    power_plugged = message['power_plugged']
    #Get the timeseries names
    mac_battery = f"{mac_address}:battery"
    mac_power = f"{mac_address}:power"
    #If the timeseries does not exists,
    # create it
    if not redis_client.ts().execute_command(f'EXISTS {mac_battery}'):
        redis_client.ts().create(mac_battery, uncompressed = False, chunk_size=128)                             #timeseries for battery level \in [0,100]
        redis_client.ts().create(mac_power, uncompressed = False, chunk_size=128)
    #Store the data on redis
    redis_client.ts().add(mac_battery, timestamp= timestamp_in_ms, value= battery_level)
    redis_client.ts().add(mac_power, timestamp= timestamp_in_ms, value= power_plugged) 
    

In [5]:
#Bind the callbacks with the client
client.on_connect = on_connect
client.on_message = on_message

In [6]:
#Connect the client to the Message Broker
client.connect(
    host = 'mqtt.eclipseprojects.io',
    port = 1883
)

0

In [7]:
#Listen to infinity
client.loop_forever()

### For debug purpose

In [8]:
mac_address = '0x71685e74f197'
mac_battery = f"{mac_address}:battery"
mac_power = f"{mac_address}:power"

print(mac_battery)

#Check if the data are stored
#print(f"Redis battery record: {redis_client.keys(match='0x([0-9a-fA-F]{12})')}")
print(f'Redis power record: {redis_client.ts().get(mac_power)}')

0x71685e74f197:battery
Redis power record: (1673697034650, 0.0)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=81f45795-723f-4142-b302-f9037f9c1cf7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>